- star detection
    - convert to grayscale
        + and blur (for better robustness?)
        - convert to float32, and divide by maximum value
    + [ ] remove hot pixel (median filter)
    - wavelet transform
        - noise reduction (remove small-scale wavelet layer)
        - remove background (remove large-scale wavelet layer)
        - inverse wavelet transform
    - detect
        - threshold image to binary
            - [ ] auto tune threshold paremeter
        - find contours
            - outer boundary: relative brightness with local background (want relatively large boundary to include all pixels for intensity centroid calculation)
            - filter by brightness
                - too dim: peak brightness
                - [ ] capped brightness (not so useful in star matching)
            - [ ] filter non-circular object: shape (`cv.minAreaRect()`)
        - characterization
            - find contour centroid
                - calculate actual intensity centroid
            - denote brightness
- star match
    - star field structure
- star align
    - calculate transform matrix

In [1]:
from __future__ import annotations

In [2]:
import os

In [3]:
# set tiff input folder
dir_input_tiff = os.path.normpath(os.path.join(
    os.getcwd(),
    '../', 'input_tiff/size_full',
))

# sort input files
tiff_list = os.listdir(dir_input_tiff)
tiff_list.sort()
# take the sequence middle as the reference frame
reference_tiff = tiff_list[len(tiff_list) // 2]
# and remove it from the 'to align' list
tiff_list.pop(len(tiff_list) // 2)

'__T_0523.TIF'

In [4]:
import numpy as np
import cv2 as cv

In [5]:
# read tiff file into numpy array using opencv
refe = cv.imread(
    os.path.join(dir_input_tiff, reference_tiff),
    cv.IMREAD_UNCHANGED,
)

## Star Detection

### convert to grayscale

In [6]:
refe_gray = cv.cvtColor(refe, cv.COLOR_BGR2GRAY)  # grayscale
refe_blur = cv.GaussianBlur(refe_gray, (9, 9), 0, 0)  # blur

refe_b_float = refe_blur.astype(np.float32) / np.iinfo(refe_blur.dtype).max
refe_float = refe_gray.astype(np.float32) / np.iinfo(refe_gray.dtype).max

### wavelet transform

In [7]:
import pywt

In [8]:
def wavelet_dec_red_rec(
    image: np.ndarray,
    level: int = 5,
    remove_to_small_scale_layer: int = -1,
    remove_large_scale: bool = True,
) -> np.ndarray:
    # decomposition
    coeffs = pywt.wavedec2(image, 'db8', level=level)
    # reduction
    for n in tuple(range(remove_to_small_scale_layer, 0)):
        for i in tuple(range(0, 3)):
            coeffs[n][i].fill(0)
    if remove_large_scale:
        coeffs[0].fill(0)
    # reconstruction
    return pywt.waverec2(coeffs, 'db8')

In [9]:
refe_b_wlred = wavelet_dec_red_rec(refe_b_float)
refe_wlred = wavelet_dec_red_rec(refe_float)

### detect

In [10]:
# threshold
refe_b_binary = cv.threshold(
    (t := refe_b_wlred),
    t.min() + (t.max() - t.min()) * 0.4,
    255,
    cv.THRESH_BINARY,
)[1].astype(np.uint8)

# find contours
refe_contours = cv.findContours(
    refe_b_binary, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE
)[0]

#### characterization

In [11]:
refe_stars: list[tuple[tuple[float, float], float]] = []  # [( (x, y), intensity )]

for c in refe_contours:
    # calculate the centroid from clear image, to increase precision
    star_mask = cv.drawContours(
        np.zeros(refe_wlred.shape, refe_wlred.dtype),
        [c], 0, 1, cv.FILLED,
    )
    M = cv.moments(refe_wlred * star_mask)
    centroid = ( M['m10'] / M['m00'], M['m01'] / M['m00'] )
    # denote brightness from blur image, to increase robustness
    brightness = refe_b_wlred[int(centroid[1]), int(centroid[0])]
    
    refe_stars.append((centroid, brightness))

#### filter and sort by brightness

In [12]:
refe_stars.sort(key=lambda e: e[1], reverse=True)
mean = np.array(tuple(e[1] for e in refe_stars)).mean()
for i in range(len(refe_stars)):
    if refe_stars[i][1] < mean:
        refe_stars = refe_stars[:i]
        break

## Star Match

### star field structure

## Image Debug Area

In [13]:
import matplotlib.pyplot as plt

In [14]:
def cvshow(name: str, image: np.ndarray):
    cv.namedWindow(name, cv.WINDOW_NORMAL)
    cv.resizeWindow(name, 1000, 1000)
    cv.moveWindow(name, 130, 20)
    cv.imshow(name, image)

In [ ]:
timg = cv.cvtColor(
    (refe_gray * (np.iinfo(np.uint8).max / np.iinfo(np.uint16).max)).astype(np.uint8),
    cv.COLOR_GRAY2BGR,
)
cv.drawContours(timg, refe_contours, -1, (0, 0, 255), cv.FILLED)

In [16]:
timg = cv.cvtColor(
    (refe_gray * (np.iinfo(np.uint8).max / np.iinfo(np.uint16).max)).astype(np.uint8),
    cv.COLOR_GRAY2BGR,
)
for c in tuple(e[0] for e in refe_stars):
    cv.drawMarker(timg, tuple(int(e) for e in c), (0, 0, 255))

In [17]:
# cvshow('refe_photo', refe_photo)
# cvshow('refe_photo_g', refe_photo_g)
# [1844:2044, 1844:2044]
# [1744:2144, 1744:2144]
# [1444:2444, 1444:2444]


cvshow('gray', refe_gray[2500:2550, 125:175])
cvshow('wlred', refe_wlred[2500:2550, 125:175])
cvshow('timg', timg[2500:2550, 125:175])


cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
cv.waitKey(0)
cv.destroyAllWindows()